In [ ]:
import spinSimulation as ss
import numpy as np
import scipy.linalg as spla
import importlib
from tqdm.auto import trange
import matplotlib.pyplot as plt

In [ ]:
N = 4
dim = 2**N

# pulse = .25e-6    # duration of pulse
# delay = 3e-6      # duration of delay
coupling = 5e3    # coupling strength
delta = 500       # chemical shift strength (for identical spins)

(x,y,z) = (ss.x, ss.y, ss.z)
(X,Y,Z) = ss.getTotalSpin(N, dim)

Hdip, Hint = ss.getAllH(N, dim, coupling, delta)
HWHH0 = ss.getHWHH0(X,Y,Z,delta)

Some quick maths, it takes about 250 seconds to make 40x40x40 = 64,000 calculations. That's 256 calculations per second.

In [ ]:
numSamples = 10
numDipCoupling = 50
pulses = np.geomspace(1e-9,1e-6, numSamples)
delays = np.geomspace(1e-7,1e-4, numSamples)

fMat = np.zeros((numSamples, numSamples))
rewardMat = np.zeros((numSamples, numSamples))
for i in trange(numSamples):
    p = pulses[i]
    for j in range(numSamples):
        d = delays[j]
        fTot = 0.0
        rTot = 0.0
        for k in range(numDipCoupling):
            _, Hint = ss.getAllH(N, dim, coupling, delta)
            UWHH = ss.getUWHH(Hint, d, p, X, Y)
            UWHH0 = ss.getPropagator(HWHH0, 6*d + 4*p)
            f = ss.fidelity(UWHH, UWHH0)
            fTot += f
            rTot += -np.log10(1 + 1e-12 - np.minimum(f, 1))
        fMat[i,j] = fTot / numDipCoupling
        rewardMat[i,j] = rTot / numDipCoupling

In [ ]:
%matplotlib inline

X1,Y1 = np.meshgrid(pulses, delays)

plt.pcolormesh(X1, Y1, rewardMat.T)
plt.title('Rewards vs pulse, delay durations')
plt.colorbar()
plt.xlabel('Pulse length')
plt.ylabel('Delay length')
plt.xscale('log')
plt.yscale('log')